In [41]:
import pandas as pd
import numpy as np
import os
import gc
import sklearn as skl
import matplotlib.pyplot as plt
import seaborn as sns
os.chdir('/home/sambeet/data/AV/am_expert/')

In [42]:
historical = pd.read_csv('train_amex/historical_user_logs.csv',parse_dates=['DateTime'],index_col='DateTime').sort_index()
historical.columns = ['user_id','product_id','action']
historical = pd.get_dummies(data=historical,columns=['action'])

In [43]:
z = x[['user_id','product_id','pct_interest_up']]
def pd(x):
    if x > 0:
        return 1
    else:
        return 0
z['pct_interest_up'] = z.pct_interest_up.apply(pd)
z = z.pivot(index = 'user_id',columns='product_id',values='pct_interest_up').reset_index().fillna(0)
z.to_csv('hist_user_id_product_held.csv',index=False)

/home/sambeet/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [44]:
#####Interest Features#####
#User-product
x = historical.groupby(['user_id','product_id']).agg({'action_interest':['count','sum'],'action_view':'sum'}).reset_index()
x['pct_interest_up'] = x.action_interest['sum']/x.action_interest['count']
x['pct_view_up'] = x.action_view['sum']/x.action_interest['count']
x['clicks_up'] = x.action_interest['count']
x.drop(['action_interest','action_view'],axis=1,inplace=True)
x.to_csv('hist_up.csv',index=False)

#x = x.merge(y,on='product_id',how='left')
#x = x.merge(z,on='user_id',how='left')

In [46]:
#####User-time features#####
#User
z = historical.groupby(['user_id']).agg({'action_interest':['count','sum'],'action_view':'sum'}).reset_index()
z['pct_interest_u'] = z.action_interest['sum']/z.action_interest['count']
z['pct_view_u'] = z.action_view['sum']/z.action_interest['count']
z['clicks_u'] = z.action_interest['count']
z.drop(['action_interest','action_view'],axis=1,inplace=True)
z.to_csv('hist_u.csv',index=False)
days_active = historical.reset_index().groupby(['user_id'])['DateTime'].agg(lambda x: (x.max() - x.min()).days if (x.max() - x.min()).days !=0 else 1)
unique_days_active = historical.reset_index().groupby(['user_id'])['DateTime'].agg(lambda x: len(np.unique(x.dt.dayofyear)))
user_time_features = days_active.reset_index().merge(unique_days_active.reset_index(),on='user_id',how = 'left')
user_time_features.columns = ['user_id','days_active','unique_days_active']
user_time_features.to_csv('hist_ut.csv',index=False)

x.columns = ['user_id','product_id','pct_interest_up','pct_view_up','clicks_up','pct_interest_p','pct_view_p','clicks_p','pct_interest_u','pct_view_u','clicks_u']
hist_features = x.merge(user_time_features,on='user_id',how='left')

In [47]:
#Product
y = historical.groupby(['product_id']).agg({'action_interest':['count','sum'],'action_view':'sum'}).reset_index()
y['pct_interest_p'] = y.action_interest['sum']/y.action_interest['count']
y['pct_view_p'] = y.action_view['sum']/y.action_interest['count']
y['clicks_p'] = y.action_interest['count']
y.drop(['action_interest','action_view'],axis=1,inplace=True)
y.to_csv('hist_p.csv',index=False)